# Deep Learning

This notebook serves as the supporting material for the chapter **Deep Learning**. In this notebook, we'll learn different activation funtions. Then we'll create a deep neural network using Deeplearning4j and train a model capable of classifying random handwriting digits. 

>_"While handwriting recognition has been attempted by different machine learning algorithms over the years, deep learning performs remarkably well and achieves an accuracy of over 99.7% on the MNIST dataset."_ 

So, let's begin...

In [1]:
%%classpath add mvn
org.nd4j nd4j-native-platform 0.9.1
org.deeplearning4j deeplearning4j-core 0.9.1
org.datavec datavec-api 0.9.1
org.datavec datavec-local 0.9.1
org.datavec datavec-dataframe 0.9.1
org.bytedeco javacpp 1.5

## Vanishing/Exploding Gradients Problem

In [2]:
import org.nd4j.linalg.api.ndarray.INDArray;
import org.nd4j.linalg.factory.Nd4j;
import org.nd4j.linalg.ops.transforms.Transforms;
import org.nd4j.linalg.api.iter.NdIndexIterator;

import java.util.ArrayList;
import java.util.List;

INDArray array = Nd4j.linspace(-5,5,200);
INDArray sigmoid = Transforms.sigmoid(array);

def ch = new Crosshair(color: Color.gray, width: 2, style: StrokeType.DOT);
p1 = new Plot(title: "Sigmoid activation function", crosshair: ch);
p1 << new ConstantLine(x: 0, y: 0, color: Color.black);
p1 << new ConstantLine(y: 1, color: Color.black, style: StrokeType.DOT);
p1 << new Line(x: [-5, 5], y: [-3/4, 7/4], style: StrokeType.DASH, color: Color.green);
p1 << new Line(x: toDoubleArrayList(array), y: toDoubleArrayList(sigmoid), displayName: "Sigmoid", color: Color.blue, width: 3);
p1 << new Text(x: 0, y: 0.5, text: "Linear", pointerAngle: 3.505);
p1 << new Text(x: -5, y: 0, text: "Saturating", pointerAngle: 1.57);
p1 << new Text(x: 5, y: 1, text: "Saturating", pointerAngle: 4.71);

public List<Double> toDoubleArrayList(INDArray array){
    NdIndexIterator iter = new NdIndexIterator(200);
    List<Double> list = new ArrayList<Double>();
    while (iter.hasNext()) {
        int[] nextIndex = iter.next();
        double nextVal = array.getDouble(nextIndex);
        list.add(nextVal);
    }
    return list;
}

## Nonsaturating Activation Functions

###  1.) Leaky ReLU

In [3]:
import org.nd4j.linalg.api.ndarray.INDArray;
import org.nd4j.linalg.factory.Nd4j;
import org.nd4j.linalg.ops.transforms.Transforms;
import org.nd4j.linalg.api.iter.NdIndexIterator;

import java.util.ArrayList;
import java.util.List;

INDArray array = Nd4j.linspace(-5,5,200);
INDArray relu = Transforms.relu(array);
INDArray leakyRelu = Transforms.leakyRelu(array);

def ch = new Crosshair(color: Color.gray, width: 2, style: StrokeType.DOT);
p1 = new Plot(title: "Leaky ReLU activation function", crosshair: ch);
p1 << new ConstantLine(x: 0, y: 0, color: Color.black);
p1.getYAxes()[0].setBound(-1,5);
p1 << new Line(x: toDoubleArrayList(array), y: toDoubleArrayList(relu), displayName: "ReLU", color: Color.orange)
p1 << new Line(x: toDoubleArrayList(array), y: toDoubleArrayList(leakyRelu), displayName: "Leaky ReLU", color: Color.blue);
p1 << new Text(x: -5, y: 0, text: "Leak", pointerAngle: 1.57);


public List<Double> toDoubleArrayList(INDArray array){
    NdIndexIterator iter = new NdIndexIterator(200);
    List<Double> list = new ArrayList<Double>();
    while (iter.hasNext()) {
        int[] nextIndex = iter.next();
        double nextVal = array.getDouble(nextIndex);
        list.add(nextVal);
    }
    return list;
}

Let's train a neural network on MNIST using the Leaky ReLU.

In [4]:
import org.datavec.api.io.labels.ParentPathLabelGenerator;
import org.datavec.api.split.FileSplit;
import org.datavec.image.loader.NativeImageLoader;
import org.datavec.image.recordreader.ImageRecordReader;
import org.deeplearning4j.datasets.datavec.RecordReaderDataSetIterator;
import org.deeplearning4j.eval.Evaluation;
import org.deeplearning4j.nn.api.OptimizationAlgorithm;
import org.deeplearning4j.nn.conf.MultiLayerConfiguration;
import org.deeplearning4j.nn.conf.NeuralNetConfiguration;
import org.deeplearning4j.nn.conf.Updater;
import org.deeplearning4j.nn.conf.inputs.InputType;
import org.deeplearning4j.nn.conf.layers.DenseLayer;
import org.deeplearning4j.nn.conf.layers.OutputLayer;
import org.deeplearning4j.nn.multilayer.MultiLayerNetwork;
import org.deeplearning4j.nn.weights.WeightInit;
import org.deeplearning4j.optimize.listeners.ScoreIterationListener;
import org.nd4j.linalg.activations.Activation;
import org.nd4j.linalg.dataset.api.iterator.DataSetIterator;
import org.nd4j.linalg.dataset.api.preprocessor.DataNormalization;
import org.nd4j.linalg.dataset.api.preprocessor.ImagePreProcessingScaler;
import org.nd4j.linalg.lossfunctions.LossFunctions;
import org.slf4j.Logger;
import org.slf4j.LoggerFactory;

import java.io.File;
import java.util.Random;

int seed = 123;
double learningRate = 0.01;
int batchSize = 100;
int numEpochs = 3;

int height = 28;
int width = 28;
int numInput = height * width;
int numHidden = 1000;
int numOutput = 10;


//Prepare data for loading
File trainData = new File("./assets/mnist_png/training");
FileSplit trainSplit = new FileSplit(trainData, NativeImageLoader.ALLOWED_FORMATS, new Random(seed));
ParentPathLabelGenerator labelMaker = new ParentPathLabelGenerator(); // use parent directory name as the image label
ImageRecordReader trainRR = new ImageRecordReader(height, width, 1, labelMaker);
trainRR.initialize(trainSplit);
DataSetIterator trainIter = new RecordReaderDataSetIterator(trainRR, batchSize, 1, numOutput);
DataNormalization imageScaler = new ImagePreProcessingScaler();
imageScaler.fit(trainIter);
trainIter.setPreProcessor(imageScaler);


File testData = new File("./assets/mnist_png/testing");
FileSplit testSplit = new FileSplit(testData, NativeImageLoader.ALLOWED_FORMATS, new Random(seed));
ImageRecordReader testRR = new ImageRecordReader(height, width, 1, labelMaker);
testRR.initialize(testSplit);
DataSetIterator testIter = new RecordReaderDataSetIterator(testRR, batchSize, 1, numOutput);
testIter.setPreProcessor(imageScaler);

//Build the neural network
MultiLayerConfiguration conf = new NeuralNetConfiguration.Builder()
        .seed(seed)
        .optimizationAlgo(OptimizationAlgorithm.STOCHASTIC_GRADIENT_DESCENT)
        .updater(Updater.ADAM)
        .list()
        .layer(0, new DenseLayer.Builder()
                .nIn(numInput)
                .nOut(numHidden)
                .activation(Activation.RELU)
                .weightInit(WeightInit.XAVIER)
                .build())
        .layer(1, new OutputLayer.Builder(LossFunctions.LossFunction.NEGATIVELOGLIKELIHOOD)
                .nIn(numHidden)
                .nOut(numOutput)
                .activation(Activation.SOFTMAX)
                .weightInit(WeightInit.XAVIER)
                .build())
        .setInputType(InputType.convolutional(28, 28, 1))
        .build();

MultiLayerNetwork model = new MultiLayerNetwork(conf);
model.init();

//Train the model and evaluate
System.out.println("Evaluation Stats:");
for (int i = 0; i < numEpochs; i++) {
    model.fit(trainIter);
    System.out.println("Completed epoch " + i);
    Evaluation eval = model.evaluate(testIter);
    System.out.println(eval.stats());

    trainIter.reset();
    testIter.reset();
}

System.out.println("********Example finished*********");

Evaluation Stats:
Completed epoch 0

Examples labeled as 0 classified by model as 0: 960 times
Examples labeled as 0 classified by model as 2: 3 times
Examples labeled as 0 classified by model as 3: 2 times
Examples labeled as 0 classified by model as 4: 2 times
Examples labeled as 0 classified by model as 5: 1 times
Examples labeled as 0 classified by model as 6: 2 times
Examples labeled as 0 classified by model as 7: 5 times
Examples labeled as 0 classified by model as 8: 2 times
Examples labeled as 0 classified by model as 9: 3 times
Examples labeled as 1 classified by model as 1: 1124 times
Examples labeled as 1 classified by model as 2: 2 times
Examples labeled as 1 classified by model as 3: 1 times
Examples labeled as 1 classified by model as 4: 1 times
Examples labeled as 1 classified by model as 5: 1 times
Examples labeled as 1 classified by model as 6: 3 times
Examples labeled as 1 classified by model as 8: 3 times
Examples labeled as 2 classified by model as 0: 3 times
Exampl

Completed epoch 2

Examples labeled as 0 classified by model as 0: 971 times
Examples labeled as 0 classified by model as 1: 1 times
Examples labeled as 0 classified by model as 2: 1 times
Examples labeled as 0 classified by model as 3: 1 times
Examples labeled as 0 classified by model as 4: 1 times
Examples labeled as 0 classified by model as 6: 1 times
Examples labeled as 0 classified by model as 8: 3 times
Examples labeled as 0 classified by model as 9: 1 times
Examples labeled as 1 classified by model as 1: 1129 times
Examples labeled as 1 classified by model as 2: 1 times
Examples labeled as 1 classified by model as 3: 1 times
Examples labeled as 1 classified by model as 6: 2 times
Examples labeled as 1 classified by model as 8: 2 times
Examples labeled as 2 classified by model as 0: 2 times
Examples labeled as 2 classified by model as 1: 5 times
Examples labeled as 2 classified by model as 2: 1001 times
Examples labeled as 2 classified by model as 3: 6 times
Examples labeled as 2

null